# IIC-3800 Tópicos en CC - NLP UC

In [44]:
class InvertedIndex:

    def __init__(self):
        self.index = dict()

    def __contains__(self, item):
        return item in self.index

    def __getitem__(self, item):
        return self.index[item]

    def add(self, word, docid):
        if word in self.index:
            if docid in self.index[word]:
                self.index[word][docid] += 1
            else:
                self.index[word][docid] = 1
        else:
            d = dict()
            d[docid] = 1
            self.index[word] = d

    #frequency of word in document, Tf
    def get_document_frequency(self, word, docid):
        if word in self.index:
            if docid in self.index[word]:
                return self.index[word][docid]
            else:
                raise LookupError('%s not in document %s' % (str(word), str(docid)))
        else:
            raise LookupError('%s not in index' % str(word))

    #number of documents that contain word, ni
    def get_index_frequency(self, word):
        if word in self.index:
            return len(self.index[word])
        else:
            raise LookupError('%s not in index' % word)


class DocumentLengthTable:

    def __init__(self):
        self.table = dict()

    def __len__(self):
        return len(self.table)

    def add(self, docid, length):
        self.table[docid] = length

    def get_length(self, docid): #dl
        if docid in self.table:
            return self.table[docid]
        else:
            raise LookupError('%s not found in table' % str(docid))

    def get_average_length(self): #avgdl
        sum = 0 
        for length in self.table.values():
            sum += length
        return float(sum) / float(len(self.table))


def build_data_structures(corpus):
    idx = InvertedIndex()
    dlt = DocumentLengthTable()
    for docid in corpus:

        #build inverted index, sequential scan
        for word in corpus[docid]:
            idx.add(str(word), str(docid))

        #build document length table
        length = len(corpus[str(docid)])
        dlt.add(docid, length)
    return idx, dlt

In [45]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer

# Load stop-words
stop_words = set(stopwords.words('english'))

# Initialize tokenizer
# It's also possible to try with a stemmer or to mix a stemmer and a lemmatizer
tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

def tokenize(document):
    words = []

    for sentence in sent_tokenize(document):
        tokens = [lemmatizer.lemmatize(t.lower()) for t in tokenizer.tokenize(sentence) if t.lower() not in stop_words and len(t) > 2]
        words += tokens

    return words


In [46]:
import re


class CorpusParser:

    def __init__(self, filename):
        self.filename = filename
        self.regex = re.compile('^#\s*\d+')
        self.corpus = dict()

    def parse(self):
        with open(self.filename) as f:
            s = ''.join(f.readlines())
        blobs = s.split('#')[1:]
        for doc in blobs:
            text = doc.split()
            docid = text.pop(0)
            words = tokenize(doc)
            self.corpus[docid] = words

    def get_corpus(self):
        return self.corpus


In [47]:
from math import log

k1 = 1.2
k2 = 100
b = 0.75
R = 0.0


def score_BM25(n, f, qf, r, N, dl, avdl):
    K = compute_K(dl, avdl)
    first = log( ( (r + 0.5) / (R - r + 0.5) ) / ( (n - r + 0.5) / (N - n - R + r + 0.5)) )
    second = ((k1 + 1) * f) / (K + f)
    third = ((k2+1) * qf) / (k2 + qf)
    return first * second * third


def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)) )


In [53]:
def run_query(index, dlt, query):
        query_result = dict()
        query_words = tokenize(query)
        for term in query_words:
            if term in index:
                doc_dict = index[term] # retrieve index entry
                for docid, freq in doc_dict.items(): #for each document and its word frequency
                    score = score_BM25(n=len(doc_dict), f=freq, qf=1, r=0, N=len(dlt), dl=dlt.get_length(docid), avdl=dlt.get_average_length()) # calculate score
                    if docid in query_result: #this document has already been scored once
                        query_result[docid] += score
                    else:
                        query_result[docid] = score
        return query_result
    

In [54]:
cp = CorpusParser(filename='corpus.txt')
cp.parse()
corpus = cp.get_corpus()

In [55]:
index, dlt = build_data_structures(corpus)

In [59]:
results = run_query(index, dlt, 'technical reports of the acm')

In [64]:
sorted_results = dict(sorted(results.items(), key=lambda item:item[1], reverse=True))

In [65]:
corpus['1086']

['propos',
 'input',
 'output',
 'convent',
 'algol',
 'report',
 'subcommitte',
 'algol',
 'acm',
 'program',
 'languag',
 'committe',
 'cacm',
 'mai',
 'march']